In [ ]:
## download all by 1 ZONE 1
import requests
from bs4 import BeautifulSoup
import os
import json
import pprint as pp
import time
from random import randint
from time import sleep
import urllib.request 

ZONE1_path = r'./ZONE1/'
ZONE2_path = r'./ZONE2/'
ZONE3_path = r'./ZONE3/'
if not os.path.exists(ZONE1_path):
    os.mkdir(ZONE1_path)
if not os.path.exists(ZONE2_path):
    os.mkdir(ZONE2_path)
if not os.path.exists(ZONE3_path):
    os.mkdir(ZONE3_path)
    
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'} 

url_start = 'https://twtmsearch.tipo.gov.tw/OS0/OS0401_q.jsp?tid=5f1da434-ad44-4955-97d3-1ac4433603f1(203.204.81.115)'
url_start = 'https://twtmsearch.tipo.gov.tw/OS0/OS0401_q.jsp?tid=f61d5b9e-0eb7-4160-86c9-ab76e064cb8c(111.249.61.96)'
url_each = 'https://twtmsearch.tipo.gov.tw/OS0/OS0401_q.jsp'
dkind = ["OPP","VAL","REV"] # "REJ" over 130K
paths = [ZONE1_path,ZONE2_path,ZONE3_path]
years = [102,103,104,105,106,107,108,109]
ZONE1_targets_fore = [399,349,384,359,307,386,384,273]
ZONE1_targets_back = [496,410,374,381,342,429,358,192]
ZONE2_targets_fore = [134,101,95,96,77,88,101,78]
ZONE2_targets_back = [174,156,139,97,101,125,79,53]
ZONE3_targets_fore = [238,261,257,277,251,277,237.307]
ZONE3_targets_back = [222,261,353,279,261,272,273,303]
keys = ["title","kind","markname","url_images","content"]

for num in range(2,3):
#select type of judgement.
    print("now is : kind",dkind[num])

    # year period select
    for year in range(8):
        try:
            data_fore = {'state': 'queryResultJSONObject',
            'searchConditions': '{"datePubStart":"%s/01/01","datePubEnd":"%s/06/31","dptKind":"%s","userType":"=====請選擇=====","boolean1":"=====請選擇=====","boolean2":"=====請選擇====="}'%(years[year],years[year],dkind[num])
                   }
            data_back = {'state': 'queryResultJSONObject',
            'searchConditions': '{"datePubStart":"%s/07/01","datePubEnd":"%s/12/31","dptKind":"%s","userType":"=====請選擇=====","boolean1":"=====請選擇=====","boolean2":"=====請選擇====="}'%(years[year],years[year],dkind[num])
                   }
            res_fore = requests.post(url_start, headers=headers, data=data_fore)
            res_back = requests.post(url_start, headers=headers, data=data_back)
            
            list_fore = json.loads(res_fore.text)
            list_back = json.loads(res_back.text)
            item_back = 0
            item_fore = 0
            
            # crawler fore year data
            for i in list_fore['receiveData']['resultArr']:
                d1 = {}
                try:
                    kind_fore = i['DPT_KIND']
                    mark_fore = i['MARK_NAME']
                    no_fore = i['PUB_NO']
                    title_fore = no_fore[-12:-1]
#                     print(title_fore)
                    file_fore = i['FILENAME']
                    data_sfore = {'state': 'getContentByFileName',
                                'filename': file_fore
                                 }

                    # crawler content
                    res_sec_fore = requests.post(url_each, headers=headers, data=data_sfore)
                    raw_content_fore = json.loads(res_sec_fore.text)
                    content_fore = raw_content_fore['content']['content']

                    #crawler image
                    case_no = raw_content_fore['content']['main'][77:118]
                    url_img = 'https://twtmsearch.tipo.gov.tw/SS0/SS0201_q.jsp?q=1&state=doQuery&showType=2&caseType=1&caseNo=%s&caseNo2=&isReadBulletinen_US=&isReadBulletinzh_TW=true&l6=zh_TW' %(case_no)
                    res_img = requests.get(url_img, headers=headers)
                    raw_img = json.loads(res_img.text)['datas']['SCN2']['fullFileName']
                    url_images = 'https://twtmsearch.tipo.gov.tw/imageLoad.jsp?path=%s&formatName=jpeg&pathCodeId=282_pic'%(raw_img)

                    # save image.
                    if not os.path.exists('%s/%s/%s.jpg' %(paths[num],years[year],title_fore)):
                        urllib.request.urlretrieve(url_images,'%s/%s/%s.jpg' %(paths[num],years[year],title_fore))
                    
                    #key:value
                    values = [title_fore,kind_fore,mark_fore,url_images,content_fore]
                    for n in range(len(keys)):
                        d1[keys[n]]=values[n]
                    
                    #save jsonfile
                    try:
                        with open('%s/%s.json' %(paths[num],years[year]), 'a', encoding='utf-8') as f:
                            json.dump(d1, f, ensure_ascii=False)
                            f.write('\n')
                    except FileNotFoundError as e:
                        no_fore = no_fore.replace('/', '-')
                        with open('%s/%s.json' %(paths[num],years[year]), 'a', encoding='utf-8') as f:
                            json.dump(d1, f, ensure_ascii=False)
                            f.write('\n')
                    except OSError:
                        pass
                    print("save json complete!===========================================================")

                    #print result
                    item_fore +=1
                    if dkind[num] == "OPP":
                        print(dkind[num],years[year],'fore: ',item_fore,"/%s"%(ZONE1_targets_fore[year]), title_fore)
                    if dkind[num] == "VAL":
                        print(dkind[num],years[year],'fore: ',item_fore,"/%s"%(ZONE2_targets_fore[year]), title_fore)
                    if dkind[num] == "REV":
                        print(dkind[num],years[year],'fore: ',item_fore,"/%s"%(ZONE3_targets_fore[year]), title_fore)
                    
                    #sleep
                    sleep(randint(5,7))
                    
                except TypeError as e:
                    print('==========')
                    print(title_fore,': no content found')
                    print(e.args)
                    print('==========')
                    continue
                except json.JSONDecodeError as e:
                    print('==========')
                    print(title_fore, ':no page found')
                    print(e.args)
                    print('==========')
                    continue
                    
            #sleep range
            sleep(randint(10,15))

            # crawler back year data
            for j in list_back['receiveData']['resultArr']:
                d2 = {}
                try:
                    kind_back = j['DPT_KIND']
                    mark_back = j['MARK_NAME']
                    file_back = j['FILENAME']
                    no_back = j['PUB_NO']
                    title_back = no_back[-12:-1]
                    data_sback = {'state': 'getContentByFileName',
                                'filename': file_back
                                 }
                    res_sec_back = requests.post(url_each, headers=headers, data=data_sback)

                    raw_content_back = json.loads(res_sec_back.text)
                    content_back = raw_content_back['content']['content']


                    #crawler image
                    case_no2 = raw_content_back['content']['main'][77:118]
                    url_img2 = 'https://twtmsearch.tipo.gov.tw/SS0/SS0201_q.jsp?q=1&state=doQuery&showType=2&caseType=1&caseNo=%s&caseNo2=&isReadBulletinen_US=&isReadBulletinzh_TW=true&l6=zh_TW' %(case_no2)
                    res_img2 = requests.get(url_img2, headers=headers)
                    raw_img2 = json.loads(res_img2.text)['datas']['SCN2']['fullFileName']
                    url_images2 = 'https://twtmsearch.tipo.gov.tw/imageLoad.jsp?path=%s&formatName=jpeg&pathCodeId=282_pic'%(raw_img2)

                    #save image
                    if not os.path.exists('%s/%s/%s.jpg' %(paths[num],years[year],title_back)):
                        urllib.request.urlretrieve(url_images2,'%s/%s/%s.jpg' %(paths[num],years[year],title_back))

                    #key:value
                    values = [title_back,kind_back,mark_back,url_images2,content_back]
                    for n in range(len(keys)):
                        d2[keys[n]]=values[n]
                    
                    #save jsonfile
                    try:
                        with open('%s/%s.json' %(paths[num],years[year]), 'a', encoding='utf-8') as f:
                            json.dump(d2, f, ensure_ascii=False)
                            f.write('\n')
                    except FileNotFoundError as e:
                        no_back = no_back.replace('/', '-')
                        with open('%s/%s.json' %(paths[num],years[year]), 'a', encoding='utf-8') as f:
                            json.dump(d2, f, ensure_ascii=False)
                            f.write('\n')
                    except OSError:
                        pass
                    print("save json complete!===========================================================")
            
                    #print result
                    item_back +=1
                    if dkind[num] == "OPP":
                        print(dkind[num],years[year],'back: ',item_back,"/%s"%(ZONE1_targets_back[year]), title_back)
                    if dkind[num] == "VAL":
                        print(dkind[num],years[year],'back: ',item_back,"/%s"%(ZONE2_targets_back[year]), title_back)
                    if dkind[num] == "REV":
                        print(dkind[num],years[year],'back: ',item_back,"/%s"%(ZONE3_targets_back[year]), title_back)
                    
                    #sleep
                    sleep(randint(5,7))
                    
                except TypeError as e:
                    print('==========')
                    print(title_back,': no content found')
                    print(e.args)
                    print('==========')
                    continue
                except json.JSONDecodeError as e:
                    print('==========')
                    print(title_back, ':no page found')
                    print(e.args)
                    print('==========')
                    continue
            
            #sleep range
            sleep(randint(10,15))

        except json.JSONDecodeError as e:
            print('==========')
    #         print(res_fore.text)
            print(e.args)
            print('==========')
            continue
# sleep range (from 1 to 7)
# print(item)
# sleep(randint(1,7))